In [1]:
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt

class CompleteExpressionTrainer:
    def __init__(self):
        self.model = None
        self.img_size = 48
        self.num_classes = 7
        self.class_names = ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']
        self.setup_directories()
        
    def setup_directories(self):
        """Create necessary directories if they don't exist"""
        print("📁 Setting up directories...")
        
        # Create main directories
        directories = ['dataset', 'models']
        
        for directory in directories:
            if not os.path.exists(directory):
                os.makedirs(directory)
                print(f"✅ Created: {directory}/")
        
        # Create expression subdirectories
        for expression in self.class_names:
            expr_path = f'dataset/{expression}'
            if not os.path.exists(expr_path):
                os.makedirs(expr_path)
                print(f"✅ Created: {expr_path}/")
    
    def check_dataset(self):
        """Check if dataset has enough images"""
        print("\n🔍 Checking dataset...")
        
        total_images = 0
        min_images = 10  # Minimum images per expression
        
        for expression in self.class_names:
            expression_path = f'dataset/{expression}'
            if not os.path.exists(expression_path):
                print(f"❌ Missing directory: {expression_path}")
                return False
            
            images = [f for f in os.listdir(expression_path) 
                     if f.lower().endswith(('.jpg', '.png', '.jpeg'))]
            count = len(images)
            total_images += count
            
            status = "✅" if count >= min_images else "⚠️ "
            print(f"   {status} {expression}: {count} images")
            
            if count < min_images:
                print(f"      ⚠️  Need at least {min_images} images, found {count}")
        
        print(f"📊 Total images: {total_images}")
        
        if total_images < min_images * self.num_classes:
            print(f"\n❌ Not enough images for training!")
            print(f"💡 Please collect at least {min_images} images per expression")
            print(f"🎥 Run: python webcam_capture_fixed.py")
            return False
        
        return True
    
    def create_model(self):
        """Create a CNN model for facial expression recognition"""
        print("🧠 Creating model architecture...")
        
        model = keras.Sequential([
            # First Convolutional Block
            layers.Conv2D(32, (3, 3), activation='relu', 
                         input_shape=(self.img_size, self.img_size, 1)),
            layers.BatchNormalization(),
            layers.MaxPooling2D(2, 2),
            layers.Dropout(0.25),
            
            # Second Convolutional Block
            layers.Conv2D(64, (3, 3), activation='relu'),
            layers.BatchNormalization(),
            layers.MaxPooling2D(2, 2),
            layers.Dropout(0.25),
            
            # Third Convolutional Block
            layers.Conv2D(128, (3, 3), activation='relu'),
            layers.BatchNormalization(),
            layers.MaxPooling2D(2, 2),
            layers.Dropout(0.25),
            
            # Classifier
            layers.Flatten(),
            layers.Dense(256, activation='relu'),
            layers.BatchNormalization(),
            layers.Dropout(0.5),
            layers.Dense(128, activation='relu'),
            layers.Dropout(0.3),
            layers.Dense(self.num_classes, activation='softmax')
        ])
        
        # Compile model
        model.compile(
            optimizer=keras.optimizers.Adam(learning_rate=0.001),
            loss='categorical_crossentropy',
            metrics=['accuracy']
        )
        
        return model
    
    def prepare_data(self):
        """Prepare training and validation data"""
        print("📊 Preparing data...")
        
        # Data augmentation for training
        train_datagen = ImageDataGenerator(
            rescale=1./255,
            rotation_range=15,
            width_shift_range=0.1,
            height_shift_range=0.1,
            horizontal_flip=True,
            zoom_range=0.2,
            brightness_range=[0.8, 1.2],
            validation_split=0.2
        )
        
        # Only rescaling for validation
        val_datagen = ImageDataGenerator(
            rescale=1./255,
            validation_split=0.2
        )
        
        # Training data
        train_generator = train_datagen.flow_from_directory(
            'dataset',
            target_size=(self.img_size, self.img_size),
            color_mode='grayscale',
            batch_size=32,
            class_mode='categorical',
            subset='training',
            shuffle=True
        )
        
        # Validation data
        validation_generator = val_datagen.flow_from_directory(
            'dataset',
            target_size=(self.img_size, self.img_size),
            color_mode='grayscale',
            batch_size=32,
            class_mode='categorical',
            subset='validation',
            shuffle=True
        )
        
        print(f"✅ Training samples: {train_generator.samples}")
        print(f"✅ Validation samples: {validation_generator.samples}")
        
        return train_generator, validation_generator
    
    def train(self, epochs=25):
        """Train the facial expression model"""
        print("🎭 COMPLETE FACIAL EXPRESSION TRAINING")
        print("=" * 50)
        
        # Check dataset first
        if not self.check_dataset():
            return False
        
        # Prepare data
        train_generator, validation_generator = self.prepare_data()
        
        # Create model
        self.model = self.create_model()
        
        print("\n📋 Model Summary:")
        self.model.summary()
        
        # Callbacks
        callbacks = [
            keras.callbacks.EarlyStopping(
                patience=10, 
                restore_best_weights=True, 
                verbose=1
            ),
            keras.callbacks.ReduceLROnPlateau(
                patience=5, 
                factor=0.5, 
                min_lr=0.00001,
                verbose=1
            ),
            keras.callbacks.ModelCheckpoint(
                'models/expression_model_best.h5',
                save_best_only=True,
                monitor='val_accuracy',
                verbose=1
            )
        ]
        
        print(f"\n🚀 Starting training for {epochs} epochs...")
        print("⏳ This may take a few minutes...")
        
        try:
            # Train model
            history = self.model.fit(
                train_generator,
                epochs=epochs,
                validation_data=validation_generator,
                callbacks=callbacks,
                verbose=1
            )
            
            # Save final model
            self.model.save('models/expression_model_final.h5')
            print("\n✅ Training completed successfully!")
            print("💾 Model saved as: models/expression_model_final.h5")
            
            # Plot results
            self.plot_results(history)
            
            return True
            
        except Exception as e:
            print(f"❌ Training error: {e}")
            return False
    
    def plot_results(self, history):
        """Plot training history and results"""
        try:
            plt.figure(figsize=(15, 5))
            
            # Accuracy plot
            plt.subplot(1, 2, 1)
            plt.plot(history.history['accuracy'], 'b-', label='Training Accuracy', linewidth=2)
            plt.plot(history.history['val_accuracy'], 'r-', label='Validation Accuracy', linewidth=2)
            plt.title('Model Accuracy', fontsize=14, fontweight='bold')
            plt.xlabel('Epoch', fontsize=12)
            plt.ylabel('Accuracy', fontsize=12)
            plt.legend()
            plt.grid(True, alpha=0.3)
            
            # Loss plot
            plt.subplot(1, 2, 2)
            plt.plot(history.history['loss'], 'b-', label='Training Loss', linewidth=2)
            plt.plot(history.history['val_loss'], 'r-', label='Validation Loss', linewidth=2)
            plt.title('Model Loss', fontsize=14, fontweight='bold')
            plt.xlabel('Epoch', fontsize=12)
            plt.ylabel('Loss', fontsize=12)
            plt.legend()
            plt.grid(True, alpha=0.3)
            
            plt.tight_layout()
            plt.savefig('models/training_results.png', dpi=300, bbox_inches='tight')
            print("📊 Training results saved as: models/training_results.png")
            
            # Show final metrics
            final_train_acc = history.history['accuracy'][-1]
            final_val_acc = history.history['val_accuracy'][-1]
            final_train_loss = history.history['loss'][-1]
            final_val_loss = history.history['val_loss'][-1]
            
            print(f"\n📈 Final Metrics:")
            print(f"   Training Accuracy: {final_train_acc:.4f}")
            print(f"   Validation Accuracy: {final_val_acc:.4f}")
            print(f"   Training Loss: {final_train_loss:.4f}")
            print(f"   Validation Loss: {final_val_loss:.4f}")
            
        except Exception as e:
            print(f"⚠️  Could not plot results: {e}")

def main():
    print("🎭 COMPLETE FACIAL EXPRESSION TRAINER")
    print("=" * 50)
    
    # Create trainer instance
    trainer = CompleteExpressionTrainer()
    
    # Start training
    success = trainer.train(epochs=20)
    
    if success:
        print("\n🎉 TRAINING COMPLETED SUCCESSFULLY!")
        print("\n🚀 Next step: Run real-time detection")
        print("   python main_expression_detection.py")
    else:
        print("\n❌ Training failed. Please check the errors above.")

if __name__ == "__main__":
    main()

🎭 COMPLETE FACIAL EXPRESSION TRAINER
📁 Setting up directories...
🎭 COMPLETE FACIAL EXPRESSION TRAINING

🔍 Checking dataset...
   ✅ angry: 4495 images
   ✅ disgust: 936 images
   ✅ fear: 4597 images
   ✅ happy: 7715 images
   ✅ neutral: 5465 images
   ✅ sad: 5330 images
   ✅ surprise: 3671 images
📊 Total images: 32209
📊 Preparing data...
Found 25768 images belonging to 7 classes.
Found 6441 images belonging to 7 classes.
✅ Training samples: 25768
✅ Validation samples: 6441
🧠 Creating model architecture...


C:\Users\vansh\anaconda3\envs\env\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)



📋 Model Summary:


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 46, 46, 32)          │             320 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization                  │ (None, 46, 46, 32)          │             128 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 23, 23, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 23, 23, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 21, 21, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_1                │ (None, 21, 21, 64)          │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 10, 10, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 10, 10, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 8, 8, 128)           │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_2                │ (None, 8, 8, 128)           │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 4, 4, 128)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 4, 4, 128)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 2048)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 256)                 │         524,544 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_3                │ (None, 256)                 │           1,024 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_3 (Dropout)                  │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 128)                 │          32,896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_4 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 7)                   │             903 │
└──────────────────────────────────────┴─────────────────────────────┴──────────────

...

 Non-trainable params: 960 (3.75 KB)


🚀 Starting training for 20 epochs...
⏳ This may take a few minutes...


C:\Users\vansh\anaconda3\envs\env\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20
 12/806 ━━━━━━━━━━━━━━━━━━━━ 14:19 1s/step - accuracy: 0.1161 - loss: 3.1645

KeyboardInterrupt: 